# 🛜 Set-up para la conexión entre mysql y python 🛜

In [1]:
import json
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

1. Nos conectamos a la base de datos (script de creación en _schema_bbdd.sql_)

In [2]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
host='127.0.0.1',
database='musicstream', 
auth_plugin ='mysql_native_password')

2. Creamos el cursor

In [3]:
mycursor = cnx.cursor()

# 🏗️ Añadiendo datos desde los ficheros .csv 🏗️

### Añadimos los datos a la tabla zonas

In [4]:
query = "INSERT INTO zonas(nombre_zona) VALUES(%s)"
val = [[('mediterraneo')], [('norte')], [('centro')], [('este')]]
mycursor.executemany(query, val)
cnx.commit()

### Añadimos los datos a la tabla países

1. Definimos los diccionarios con los paises para cada zona

In [7]:
paises_mediterraneos = {
    "ES": "España",
    "FR": "Francia",
    "MC": "Mónaco",
    "IT": "Italia"}
paises_este_europa = {
    "HU": "Hungría",
    "PL": "Polonia",
    "RO": "Rumanía",
    "UA": "Ucrania"}



2. Definimos la función `añadir_pais`

In [8]:
def añadir_pais(diccionario, id_zona):
    ''' Añade datos a la tabla de paises.

        Args: 
            diccionario (dict): Un diccionario de los definidos para cada zona donde se recogen los países y sus códigos.
            id_zona (int): Número asignado a cada zona de europa en la tabla zonas. 

    '''
    for id, pais in diccionario.items():
        query = "INSERT INTO paises(id_pais, nombre_pais, id_zona) VALUES(%s, %s, %s)"
        val = [(id, pais, id_zona)] 
        mycursor.executemany(query, val)
        cnx.commit()

Los id_zona quedan entonces como: 
- 1 mediterraneo
- 2 este 

3. Llamamos a la función en bucle para añadir los países de cada zona de una sola vez. 

In [9]:
# Definimos estas dos listas, que nos van a servir para poder llamar en bucle a la función. Es MUY IMPORTANTE el orden, debe coincidir el nombre del diccionario con el id_zona asignado a cada zona.
lista_diccionarios = [paises_mediterraneos, paises_este_europa]
lista_zonas = [i for i in range(1,3)]

### Añadimos los datos a la tabla años

In [11]:
# Create the table años if it does not exist
create_table_query = """
CREATE TABLE IF NOT EXISTS años (
	id INT AUTO_INCREMENT PRIMARY KEY,
	rango_años VARCHAR(255) NOT NULL
)
"""
mycursor.execute(create_table_query)
cnx.commit()

# Insert data into the table años
query_años = "INSERT INTO años(rango_años) VALUES(%s)"
val_años = [['2000-2004'], ['2005-2009'], ['2010-2014'], ['2015-2019'], ['2020-2024']]
mycursor.executemany(query_años, val_años)
cnx.commit()

### Añadimos los datos a la tabla artistas

1. Definimos la función `convertir_csv`, que convierte cada uno de los csv que hemos creado en un DataFrame.

In [12]:
def convertir_csv(zona, años):
    ''' Convierte el csv de cada zona y rango de años en un DataFrame.
    
        Args:
            zona (str): Nombre de la zona.
            años (str): Rango de años en la forma añoinicio_añofin. 
    '''
    df = pd.read_csv(f'archivos_csv/paises_{zona}_{años}.csv',index_col = 0)
    return df

2. Llamamos a la función una vez por cada csv que debemos convertir.

In [13]:
# Definimos estas dos listas, que nos van a servir para poder llamar a la función.
lista_nombre_zona = ["este_europa", "mediterraneos", "norte_europa"]
lista_años = ["2000_2004", "2005_2009", "2010_2014", "2015_2019", "2020_2024"]

In [14]:
# Países del este
df_este_2000_2004 = convertir_csv(lista_nombre_zona[0], lista_años[0])
df_este_2005_2009 = convertir_csv(lista_nombre_zona[0], lista_años[1])
df_este_2010_2014 = convertir_csv(lista_nombre_zona[0], lista_años[2])
df_este_2015_2019 = convertir_csv(lista_nombre_zona[0], lista_años[3])
df_este_2020_2024 = convertir_csv(lista_nombre_zona[0], lista_años[4])

In [15]:
# Países mediterráneos
df_medit_2000_2004 = convertir_csv(lista_nombre_zona[2], lista_años[0])
df_medit_2005_2009 = convertir_csv(lista_nombre_zona[2], lista_años[1])
df_medit_2010_2014 = convertir_csv(lista_nombre_zona[2], lista_años[2])
df_medit_2015_2019 = convertir_csv(lista_nombre_zona[2], lista_años[3])
df_medit_2020_2024 = convertir_csv(lista_nombre_zona[2], lista_años[4])

In [16]:
lista_df_csv = [df_este_2000_2004, df_este_2005_2009, df_este_2010_2014, df_este_2015_2019, df_este_2020_2024, df_medit_2000_2004, df_medit_2005_2009, df_medit_2010_2014, df_medit_2015_2019, df_medit_2020_2024]

In [ ]:
#lista_df_ordenados = []
#for l in lista_df_csv:
    #var = l.sort_values(['popularity'], axis=0, ascending=False, ignore_index=True)
    #lista_df_ordenados.append(var)

3. Creamos una lista de artistas vacía y luego un DataFrame con todos los artistas por zona.

In [17]:
artistas = []
df_mediterraneo = pd.concat(
    [df_medit_2000_2004['artista'], df_medit_2005_2009['artista'], df_medit_2010_2014['artista'], df_medit_2015_2019['artista'], df_medit_2020_2024['artista']],
    axis=1,
    keys=["00-04", "05-09", "10-14", "15-19", "20-24"])
df_este = pd.concat(
    [df_este_2000_2004['artista'], df_este_2005_2009['artista'], df_este_2010_2014['artista'], df_este_2015_2019['artista'], df_este_2020_2024['artista']],
    axis=1,
    keys=["00-04", "05-09", "10-14", "15-19", "20-24"])

In [18]:
# Esta lista es para poder meter los datos de los df en bucle.
lista_df = [df_mediterraneo, df_este]

4. Con el siguiente bucle, convertimos cada artista en una lista y lo añadimos a la lista artistas.

In [19]:
for df in lista_df:
    for i in df.itertuples(name=None, index=False):
        for n in range(4):
            if ([i[n]]) not in artistas:
                artistas.append(([i[n]]))

In [20]:
len(artistas)

668

5. Ejecutamos la inserción 

In [21]:
query_artistas = "INSERT INTO artistas(nombre_artista) VALUES(%s)"
for artista in artistas:
    try:
     mycursor.execute(query_artistas, artista)
     cnx.commit()
    except:
      continue
cnx.close()

### Añadimos los datos a la tabla canciones 

Recordemos los id_lustro para cada conjunto de 5 años: 
- 1 = 2000-2004
- 2 = 2005-2009
- 3 = 2010-2014
- 4 = 2015-2019
- 5 = 2020-2024

Volvemos a abrir la conexión y a definir el cursor

In [22]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1',
                                    database='musicstream', 
                                    auth_plugin ='mysql_native_password')
mycursor = cnx.cursor()

Hay algunos datos que son `NaN`, y MySQL no los reconoce, por lo que hay que convertirlos a None. 

Para ello, creamos la función `eliminar_nan`

In [23]:
def eliminar_nan (df):
    df_limpio = df.replace({float('NaN'): None})
    return df_limpio

Limpiamos los df y lo metemos en la lista `lista_df_limpios` 

In [24]:
lista_df_limpios = []

for data in lista_df_csv:
    df_limpio = eliminar_nan(data)
    lista_df_limpios.append(df_limpio)
    

Definimos una función para insertar las canciones

In [25]:
def insertar_canciones (df):
    lista_querys = []
    for i in range(len(df)):
        tupla_insercion = (str(df['nombre'][i]), str(df['genero'][i]), int(df['popularity'][i]))
        lista_querys.append(tupla_insercion)
    return lista_querys

Creamos la lista de tuplas para insertar, descartando aquellas canciones que coinciden en todos los campos.

In [26]:
lista_de_tuplas = []

for df in lista_df_limpios:    
    tuplas = insertar_canciones(df)
    for t in tuplas:
        if t not in lista_de_tuplas:
            lista_de_tuplas.append(t)   
    

In [27]:
print(len(lista_de_tuplas))
print(lista_de_tuplas[:10])  # Muestra las primeras 10 tuplas


1421
[('Сукины дети', 'None', 52), ('Angel', 'None', 28), ('Sonne', 'industrial metal', 73), ('Cabelo Raspadinho', 'brasil', 2), ('Without Me', 'rap', 83), ('Devolva-Me', 'mpb', 1), ('ИСКАЛА', 'russian rock', 51), ('Dar A Volta No Brasil - Ao Vivo', 'None', 0), ('Superman', 'None', 78), ('Csak Játék', 'Ballad', 0)]


In [28]:
mycursor.execute("SELECT COUNT(*) FROM canciones;")
print(mycursor.fetchone())


(0,)


Definimos la query canciones:

In [29]:
query_canciones = "INSERT INTO canciones(cancion, genero, popularidad) VALUES(%s, %s, %s)"

In [30]:
mycursor.executemany(query_canciones, lista_de_tuplas)
cnx.commit()

In [31]:
cnx.close()

### Añadimos los datos a las tablas intermedias 

Primero, pais en cancion:

In [32]:
contador = 0
queries_pais_cancion = []
for t in lista_de_tuplas:
    contador += 1
    for df in lista_df_limpios:
        for i in range(len(df)):
            if df['nombre'][i] in t:
                tupla_pais_cancion = (df['pais'][i], contador)
                queries_pais_cancion.append(tupla_pais_cancion)
                

## EL ERROR ESTÁ AQUÍ, PROBAR, PREGUNTAR A CHAT GPT ##

In [38]:
query_pais_en_cancion = "INSERT INTO cancion_en_pais(id_cancion, id_pais) VALUES(%s, %s)"

In [34]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1',
                                    database='musicstream', 
                                    auth_plugin ='mysql_native_password')
mycursor = cnx.cursor()

In [35]:
for q in queries_pais_cancion:
    try:
        mycursor.execute(query_pais_en_cancion, q)
        cnx.commit()
    except:
        print(f"La entrada {q} está repetida")
        

La entrada ('BY', 1) está repetida
La entrada ('MD', 1) está repetida
La entrada ('UA', 1) está repetida
La entrada ('BY', 2) está repetida
La entrada ('BG', 2) está repetida
La entrada ('CZ', 2) está repetida
La entrada ('SK', 2) está repetida
La entrada ('HU', 2) está repetida
La entrada ('MD', 2) está repetida
La entrada ('PL', 2) está repetida
La entrada ('RO', 2) está repetida
La entrada ('UA', 2) está repetida
La entrada ('DK', 2) está repetida
La entrada ('EE', 2) está repetida
La entrada ('FI', 2) está repetida
La entrada ('IS', 2) está repetida
La entrada ('IE', 2) está repetida
La entrada ('LT', 2) está repetida
La entrada ('LV', 2) está repetida
La entrada ('NO', 2) está repetida
La entrada ('SE', 2) está repetida
La entrada ('GB', 2) está repetida
La entrada ('BY', 3) está repetida
La entrada ('BG', 3) está repetida
La entrada ('CZ', 3) está repetida
La entrada ('SK', 3) está repetida
La entrada ('HU', 3) está repetida
La entrada ('MD', 3) está repetida
La entrada ('PL', 3)

In [94]:
cnx.close()

Ahora, lo mismo con artista en cancion:

In [95]:
queries_artista_cancion = []
for t in lista_de_tuplas:
    for df in lista_df_limpios:
        for i in range(len(df)):
            if df['nombre'][i] in t:
                q_id_artista = f"SELECT id_artista FROM artistas WHERE nombre_artista = '{df['artista'][i]}';"
                q_id_cancion = f"SELECT id_cancion FROM canciones WHERE cancion = '{df['nombre'][i]}';"
                
                mycursor.execute(q_id_artista)
                id_artista = mycursor.fetchone()[0]
                mycursor.execute(q_id_cancion)
                id_cancion = mycursor.fetchone()[0]
                query = (id_artista, id_cancion)
                queries_artista_cancion.append(query)
                

OperationalError: 2055: Lost connection to MySQL server at '127.0.0.1:3306', system error: 10038 Se intentó realizar una operación en un elemento que no es un socket

In [ ]:
print(len(lista_de_tuplas))
print(lista_de_tuplas[:10])  # Muestra las primeras 10 tuplas


In [ ]:
queries_artista_cancion

[(20, 1)]

In [ ]:
info_artistas = f"SELECT id_artista FROM artistas WHERE nombre_artista = '{lista_df_limpios[0]['artista'][0]}';"

In [ ]:
info_canciones = f"SELECT id_artista FROM artistas WHERE cancion = '{lista_df_limpios[0]['nombre'][0]}' AND id_lustro;"

In [ ]:
mycursor.execute(info_canciones)
x = mycursor.fetchone()

InternalError: Unread result found

In [ ]:
x

20

Por último ,metemos lustro en cancion: